<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/RedditConversationQuality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Reddit Data  

#Pushshift API

In [19]:
base_path = "/content/drive/MyDrive/University/RedditData/"

##Write submmisons to file

In [20]:
import json

def filter_json_objects(data, keys):
    filtered_data = []
    for item in data:
        filtered_item = {}
        for key in keys:
            if key in item:
                filtered_item[key] = item[key]
        filtered_data.append(filtered_item)
    return filtered_data

In [21]:
def write_last_timestamp(last_timestamp, entry):
  with open(f"{base_path}last_timestamp_{entry}.txt", "w") as f:
    f.write(str(last_timestamp))

In [22]:
def read_last_timestamp(default, entry):
  try:
    with open(f"{base_path}last_timestamp_{entry}.txt", "r") as f:
        return int(f.read())
  except FileNotFoundError:
    return default

In [23]:
def write_data_to_file(data, entry):
  with open(f"{base_path}{entry}s.json", "a") as f:
    for element in data:
        json.dump(element, f)
        f.write("\n")

In [64]:
import requests
import datetime
import json
import time

def extract_date_based_data_from_reddit(entry_type, keys, start_date, end_date):
  # type can be either submission or comment
  # keys are fields from submission or comment that we want to save
  # start date and end date shows the date interval which data is collected

  step = 10

  # read the last UTC timestamp from file
  last_timestamp = read_last_timestamp(int(end_date.timestamp()), entry_type)

  # define the API endpoint
  const_url = f"https://api.pushshift.io/reddit/search/{entry_type}/?size=500&filter={','.join(keys)}&sort=created_utc&"
  url_template = const_url + "after={}&before={}"
  # get the set of enteries using pagination
  while True:
    new_url = url_template.format(
        max(last_timestamp-step,int(start_date.timestamp())),
         last_timestamp)
    try:
        new_response = requests.get(new_url)
        enteries = new_response.json()["data"]
        write_data_to_file(enteries, entry_type)
        # save the UTC timestamp of the last entry to file
        print(len(enteries))
        if len(enteries) == 0:
          last_timestamp -= step
          continue
        last_entry = enteries[-1]
        last_timestamp = last_entry["created_utc"]
        last_entry_date = datetime.datetime.fromtimestamp(last_timestamp)
        print(last_entry_date)
        write_last_timestamp(last_timestamp, entry_type)
        if last_timestamp == int(start_date.timestamp()):
          break
    except json.JSONDecodeError or requests.exceptions.HTTPError:
        if new_response.status_code == 429:
            retry_after = 30#int(new_response.headers.get("Retry-After"))
            print(f"Got HTTP error 429, waiting {retry_after} seconds and retrying...")
            time.sleep(retry_after)
            continue
        else:
            raise e

In [65]:
# define the date range
start_date = datetime.datetime(2023, 3, 1)
end_date = datetime.datetime(2023, 3, 31)
keys = ['created_utc', 'id']

extract_date_based_data_from_reddit("submission", keys, start_date, end_date)

166
2023-03-30 22:44:37
202
2023-03-30 22:44:27
209
2023-03-30 22:44:17
205
2023-03-30 22:44:07
180
2023-03-30 22:43:57
196
2023-03-30 22:43:47
174
2023-03-30 22:43:37
195
2023-03-30 22:43:27
148
2023-03-30 22:43:17
181
2023-03-30 22:43:07
170
2023-03-30 22:42:57
182
2023-03-30 22:42:47
140
2023-03-30 22:42:37
186
2023-03-30 22:42:27
172
2023-03-30 22:42:17
162
2023-03-30 22:42:07
171
2023-03-30 22:41:57
147
2023-03-30 22:41:47
184
2023-03-30 22:41:37
167
2023-03-30 22:41:27
209
2023-03-30 22:41:17
221
2023-03-30 22:41:07
151
2023-03-30 22:40:57
170
2023-03-30 22:40:47
200
2023-03-30 22:40:37
136
2023-03-30 22:40:27
249
2023-03-30 22:40:17
229
2023-03-30 22:40:07
197
2023-03-30 22:39:57
202
2023-03-30 22:39:47
195
2023-03-30 22:39:37
180
2023-03-30 22:39:27
186
2023-03-30 22:39:17
161
2023-03-30 22:39:08
149
2023-03-30 22:38:58
170
2023-03-30 22:38:48
186
2023-03-30 22:38:38
Got HTTP error 429, waiting 30 seconds and retrying...
196
2023-03-30 22:38:28
184
2023-03-30 22:38:18
192
2023-

KeyboardInterrupt: ignored

In [ ]:
import requests
import datetime
import json

# define the date range
start_date = datetime.datetime(2023, 3, 1)
end_date = datetime.datetime(2023, 3, 31)
keys = ['created_utc', 'id']

# read the last UTC timestamp from file
last_timestamp = read_last_timestamp(int(end_date.timestamp()))

# define the API endpoint
# fields not working
url_template = "https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_utc&after={}&before={}"

# get the set of submissions using pagination
while True:
    new_url = url_template.format(last_timestamp-1, last_timestamp)
    print(new_url)
    try:
        new_response = requests.get(new_url)
        submissions = new_response.json()["data"]
        write_submissions_to_file(submissions)
        # save the UTC timestamp of the last submission to file
        if len(submissions) == 0:
          last_timestamp -= 1
          continue
        if last_timestamp - 1 == int(start_date.timestamp()):
          break
        last_submission = submissions[-1]
        last_timestamp = last_submission["created_utc"]
        last_submission_date = datetime.datetime.fromtimestamp(last_submission["created_utc"])
        print(last_submission_date)
        write_last_timestamp(last_timestamp)
        
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:
            retry_after = int(e.response.headers.get("Retry-After"))
            print(f"Got HTTP error 429, waiting {retry_after} seconds and retrying...")
            time.sleep(retry_after)
            continue
        else:
            raise e

https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_utc&after=1680220758&before=1680220759
https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_utc&after=1680220757&before=1680220758
2023-03-30 23:59:17
https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_utc&after=1680220756&before=1680220757
2023-03-30 23:59:16
https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_utc&after=1680220755&before=1680220756
2023-03-30 23:59:15
https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_utc&after=1680220754&before=1680220755
2023-03-30 23:59:14
https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_utc&after=1680220753&before=1680220754
2023-03-30 23:59:13
https://api.pushshift.io/reddit/search/submission/?size=500&filter=created_utc,id&sort=created_u

KeyboardInterrupt: ignored

##Write sample submission fields to file

In [ ]:
import requests
import json

# define the API endpoint with the sample submission ID
submission_id = "1273r9g"
url = f"https://api.pushshift.io/reddit/submission/search/?ids={submission_id}"

# make the API request and get the submission
response = requests.get(url)
submission_list = response.json()["data"]

# get the first submission from the list
submission = submission_list[0]

# write the submission keys to a file
with open("submission_keys.txt", "w") as f:
    for key in submission.keys():
        f.write(key + "\n")

##Write comments to file

###With Pushshift date search

###With comment_ids Pushshift

In [ ]:
import requests
import json

# read submission IDs from file
with open("submissions.json", "r") as f:
    submissions = [json.loads(line) for line in f]

# define the API endpoint with the submission IDs
url_template = "https://api.pushshift.io/reddit/submission/comment_ids/{}"

# make the API request and get the comments
comments_data = []
for submission in submissions:
    submission_id = submission["id"]
    url = url_template.format(submission_id)
    print(url)
    response = requests.get(url)
    comment_ids = response.json()
    comment_url = f"https://api.pushshift.io/reddit/comment/search/?ids={','.join(comment_ids)}"
    comment_response = requests.get(comment_url)
    comment_data = comment_response.json()["data"]
    comments_data.extend(comment_data)
    with open("comments_data.json", "a") as f:
        json.dump(comment_data, f)
#currently not working returns empty lists

https://api.pushshift.io/reddit/submission/comment_ids/1273r9i
https://api.pushshift.io/reddit/submission/comment_ids/1273r9j
https://api.pushshift.io/reddit/submission/comment_ids/1273r9n
https://api.pushshift.io/reddit/submission/comment_ids/1273r9p
https://api.pushshift.io/reddit/submission/comment_ids/1273r9q
https://api.pushshift.io/reddit/submission/comment_ids/1273r9s
https://api.pushshift.io/reddit/submission/comment_ids/1273r9t
https://api.pushshift.io/reddit/submission/comment_ids/1273r97
https://api.pushshift.io/reddit/submission/comment_ids/1273r98
https://api.pushshift.io/reddit/submission/comment_ids/1273r9f
https://api.pushshift.io/reddit/submission/comment_ids/1273r9g
https://api.pushshift.io/reddit/submission/comment_ids/1273r9h
https://api.pushshift.io/reddit/submission/comment_ids/1273r9k
https://api.pushshift.io/reddit/submission/comment_ids/1273r9l
https://api.pushshift.io/reddit/submission/comment_ids/1273r9m
https://api.pushshift.io/reddit/submission/comment_ids/

KeyboardInterrupt: ignored

###With comment link-id

In [ ]:
import requests
import json

# read submission IDs from file
with open("submissions.json", "r") as f:
    submissions = [json.loads(line) for line in f]

# define the API endpoint with the submission IDs
url_template = "https://api.pushshift.io/reddit/comment/search/?link_id={}"

# make the API request and get the comments
comments_data = []
for submission in submissions:
    submission_id = submission["id"]
    #print(submission["num_comments"])
    url = url_template.format(submission["id"])
    print(url)
    response = requests.get(url)
    comments_data.extend(response.json()["data"])
    print(response.json()["data"])

# write the comments data to a file
with open("comments_data.json", "w") as f:
    json.dump(comments_data, f)
#Can not search by link_id

https://api.pushshift.io/reddit/comment/search/?link_id=1273r9i
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r9j
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r9n
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r9p
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r9q
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r9s
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r9t
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r97
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r98
[]
https://api.pushshift.io/reddit/comment/search/?link_id=1273r9f


KeyboardInterrupt: ignored

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Async PRAW

In [ ]:
!pip install asyncpraw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.5/196.5 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.0 MB/s eta 0:00:00


In [ ]:
import asyncpraw
import datetime

# create an async Reddit instance
reddit = asyncpraw.Reddit(
    client_id='4Fh9K7U29aRt6GUpOkoqfw',
    client_secret='tyuF_v56l2bFmSqUOXFldWPtYsFViA',
    user_agent='Simurgh__'
)

In [ ]:
# define the subreddit and date range
subreddit = await reddit.subreddit("AmItheAsshole")
await subreddit.load()  # fetch the subreddit object from Reddit
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 1, 3)
# set up pagination
params = {"before": int(end_date.timestamp()), "after": int(start_date.timestamp()), "limit": 100}
print("start ",str(start_date.timestamp()))
print("end ", str(end_date.timestamp()))
start = 0
while start<1:
  submissions = subreddit.search("*", params=params)
  print("here")
  # save submissions to a file
  with open("submissions.txt", "w") as f:
    async for submission in submissions:
        f.write(str(submission.created_utc) + "\n")
  start += 1

start  1640995200.0
end  1641168000.0
here


#PRAW

##Installation

In [ ]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Imports

In [ ]:
import praw

##Hyper Parameters

In [ ]:
ALL_SUBREDDIT = 'all'
SAMPLE_SUBREDDIT = 'MachineLearning'

##Creating Reddit Instance

In [ ]:
reddit = praw.Reddit(client_id='4Fh9K7U29aRt6GUpOkoqfw',
                     client_secret='tyuF_v56l2bFmSqUOXFldWPtYsFViA',
                     user_agent='Simurgh__')

##Getting Posts

In [ ]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = reddit.subreddit(ALL_SUBREDDIT).hot(limit=10)

In [ ]:
for post in hot_posts:
    print(post.title)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Building a hobby-shelter while camping in Kelowna
meirl
A former bonded child labourer from Pakistan escaped the carpet factory at the age of 10, helped free over 3000 kids from slavery, helped bring down Pakistani carpet exports by $34 million, before being murdered at the age of 13
Judge Frank Caprio settles 250$ worth of fines and gives 25$ to pay for guys Uber who had 92¢ in his account and walked 5 miles just so he could make it to court
Boomer Comic
Man with An American flag shirt, reaction to kids protesting gun violence in their schools.
Soap bubble freezing
Facebook and Google are handing over user data to help police prosecute abortion seekers
Gotta burst your bubble, guys. We're all getting replaced soon.
How to massage your Eagle.


In [ ]:
# get MachineLearning subreddit data
ml_subreddit = reddit.subreddit('MachineLearning')

print(ml_subreddit.description)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



**[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
--------
+[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
--------
+[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
--------
+[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
--------
+[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ANews)
--------
***[@slashML on Twitter](https://twitter.com/slashML)***
--------
***[Chat with us on Slack](https://join.slack.com/t/rml-talk/shared_invite/enQtNjkyMzI3NjA2NTY2LWY0ZmRjZjNhYjI5NzYwM2Y0YzZhZWNiODQ3ZGFjYmI2NTU3YjE1ZDU5MzM2ZTQ4ZGJmOTFmNWVkMzFiMzVhYjg)***
--------
**Beginners:**
--------
Please have a look at [our FAQ and Link-Collection](http://www.reddit.com/r/MachineLearning/wiki/index)

[Metacademy](http://www.metacademy.org) is a great resource which compiles le

Because we only have a limited amoung of requests per day it is a good idea to save the scraped data in some kind of variable or file.

In [ ]:
import pandas as pd

posts = []
ml_subreddit = reddit.subreddit('MachineLearning')
for post in ml_subreddit.hot(limit=10):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



,title,score,id,subreddit,url,num_comments,body,created
0,[D] Simple Questions Thread,10,11ckopj,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,100,Please post your questions here instead of cre...,1.677427e+09
1,[D] Video 2 Minecraft — Comparing ControlNet a...,434,11igrea,MachineLearning,https://v.redd.it/ci3jjyo42tla1,7,,1.677972e+09
2,[D] Productization of deep learning models - w...,6,11isomm,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,4,I see so many new services built with deep lea...,1.678008e+09
3,[P] LazyShell - GPT based autocomplete for zsh,708,11hscl1,MachineLearning,https://i.redd.it/amnowgji6ola1.gif,51,,1.677913e+09
4,[D] Building an Open-Source LLM Provider for S...,9,11iooio,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,2,Hi guys! \n\nI've been thinking that as of now...,1.677994e+09
5,[D] LLaMA Model Parallelization and Server Con...,3,11iul6f,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,2,"Hey everyone,\n\nFirst of all, tldr at bottom,...",1.678015e+09
6,[D] First glance at LLaMA,51,11ibm1j,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,10,[https://medium.com/@enryu9000/mini-post-first...,1.677960e+09
7,[P] diffground - A simplistic Android UI to ac...,26,11idjj6,MachineLearning,https://i.redd.it/ckkqkjyrgsla1.png,5,,1.677965e+09
8,[P] A mobile-friendly way to edit photos with ...,3,11is9ge,MachineLearning,https://v.redd.it/c3avxdjswvla1,1,,1.678006e+09
9,[D] Is it possible to run Meta's LLaMA 65B mod...,41,11i4olx,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,44,I think it's clear that a single beefy rig cou...,1.677948e+09


In [ ]:
posts.to_csv('top_ml_subreddit_posts.csv')

##Getting Posts Comments

PRAW also allows us to get information about a specifc post/submission

In [ ]:
submission = reddit.submission(url="https://www.reddit.com/r/MapPorn/comments/a3p0uq/an_image_of_gps_tracking_of_multiple_wolves_in/")
# or 
submission = reddit.submission(id="a3p0uq") #id comes after comments/

This will work for some submission, but for others that have more comments this code will throw an AttributeError saying:

``AttributeError: 'MoreComments' object has no attribute 'body'``

These MoreComments object represent the “load more comments” and “continue this thread” links encountered on the websites, as described in more detail in the comment documentation.

There get rid of the MoreComments objects, we can check the datatype of each comment before printing the body.

The below cell is another way of getting rid of the MoreComments objects

The above codeblocks only got the top lebel comments. If we want to get the complete ``CommentForest`` we need to use the ``.list`` method.

In [ ]:
submission.comments.replace_more(limit=None)
i = 0
len(submission.comments.list())
#for comment in submission.comments.list():
    #print("parent", comment.parent_id)
    #print(comment.id)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

1015